In [ ]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import cv2
import os
import warnings
import sys
sys.stdout.write("\033[1;34m")
warnings.filterwarnings("ignore")
 
train_path_im='/content/drive/My Drive/GeneralYolodata/'
dir_path_conversation = '/content/drive/My Drive/chatterbot_dataset_ita'
len_data_vqa=2000
input_im_size=64
num_epochs=2000
batch_size=128
 
#questions and answers
pre_ques = ['che cosa vedi']
pre_answers = ['vedo una camera da letto', 'vedo una città con dei palazzi', 'ci sono delle persone ',
               'una foresta con degli alberi', 'sono davanti ad una montagna', 'questo potrebbe essere un cane',
               'un gatto', 'il traffico con delle macchine', 'vedo una scrivania', 'questa dovrebbe essere una sedia']
questions = []
answers = []
 
for i in range(len_data_vqa):
    questions.append(pre_ques[0])
 
for i in range(int(len_data_vqa / 200)):
    for n in range(200):
        answers.append(pre_answers[i])
 
for filepath in os.listdir(dir_path_conversation):
    stream = open(dir_path_conversation + os.sep + filepath, 'r', encoding='utf-8')
    text = stream.readlines()
    for i in range(int(len(text) / 2) - 6):
        questions.append(text[i + i].rstrip('\n'))
        answers.append(text[i + i + 1].rstrip('\n'))
 
#images
images=[]
for i in range(len_data_vqa):
  i=i+1
  im=cv2.resize(cv2.imread(train_path_im+ str(i)+ '.jpg'),(input_im_size,input_im_size))/255.0
  images.append(im)
 
for i in range(len(questions)-len_data_vqa):
  im = cv2.resize(cv2.imread(train_path_im + 'faces_1.jpg'), (input_im_size, input_im_size)) / 255.0
  images.append(im)
 
images=np.expand_dims(images,1)
print('Images shape =',images.shape)
 
#questions
questions_lines = questions
 
tokenizer = Tokenizer()
tokenizer.fit_on_texts( questions_lines )
tokenized_eng_lines = tokenizer.texts_to_sequences( questions_lines )
 
length_list = []
for token_seq in tokenized_eng_lines:
    length_list.append( len( token_seq ))
max_input_length = np.amax( length_list )
print( 'questions max length is {}'.format( max_input_length ))
 
padded_eng_lines = pad_sequences( tokenized_eng_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( np.expand_dims(padded_eng_lines, 1))
print( 'Encoder input data shape -> {}'.format( encoder_input_data.shape ))
 
eng_word_dict = tokenizer.word_index
num_eng_tokens = len( eng_word_dict )+1
print( 'Number of questions tokens = {}'.format( num_eng_tokens))
 
#answers
answer_lines = []
lines_answers=answers
for line in range( len(lines_answers)):
    answer_lines.append( '<START> ' + str(lines_answers[line]) + ' <END>' )
 
tokenizer = Tokenizer()
tokenizer.fit_on_texts( answer_lines )
tokenized_mar_lines = tokenizer.texts_to_sequences( answer_lines )
 
length_list = []
for token_seq in tokenized_mar_lines:
    length_list.append( len( token_seq ))
max_output_length = np.amax( length_list )
print( 'Answers max length is {}'.format( max_output_length ))
 
padded_mar_lines = pad_sequences( tokenized_mar_lines , maxlen=max_output_length, padding='post' )
decoder_input_data = np.array( padded_mar_lines )
print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))
 
mar_word_dict = tokenizer.word_index
num_mar_tokens = len( mar_word_dict )+1
print( 'Number of Answers tokens = {}'.format( num_mar_tokens))
 
#decoder target data
decoder_target_data = list()
for token_seq in tokenized_mar_lines:
    decoder_target_data.append(token_seq[1:])
 
padded_mar_lines = pad_sequences(decoder_target_data, maxlen=max_output_length, padding='post')
onehot_mar_lines = to_categorical(padded_mar_lines, num_mar_tokens)
decoder_target_data = np.array(onehot_mar_lines)
print('Decoder target data shape -> {}'.format(decoder_target_data.shape))
 
#cnn model
vision_model=keras.models.Sequential()
vision_model.add(keras.layers.Conv2D(32, kernel_size=(3,3),padding='same',activation='elu',input_shape=(input_im_size,input_im_size,3)))
vision_model.add(keras.layers.Conv2D(32, kernel_size=(3,3),activation='elu'))
vision_model.add(keras.layers.MaxPooling2D((2,2)))
vision_model.add(keras.layers.Dropout(0.1))
vision_model.add(keras.layers.Conv2D(64, kernel_size=(3,3),padding='same',activation='elu'))
vision_model.add(keras.layers.Conv2D(64, kernel_size=(3,3),activation='elu'))
vision_model.add(keras.layers.MaxPooling2D((2,2)))
vision_model.add(keras.layers.Dropout(0.2))
vision_model.add(keras.layers.Conv2D(128,kernel_size=(3,3),padding='same',activation='elu'))
vision_model.add(keras.layers.Conv2D(128,kernel_size=(3,3),activation='elu'))
vision_model.add(keras.layers.Conv2D(128,kernel_size=(3,3),activation='elu'))
vision_model.add(keras.layers.MaxPooling2D((2,2)))
vision_model.add(keras.layers.Dropout(0.2))
vision_model.add(keras.layers.Flatten())
vision_model.add(keras.layers.BatchNormalization())
 
#seq2seq + cnn model
image_input=keras.layers.Input(shape=(1,input_im_size,input_im_size, 3))
cnn_lstm=keras.models.Sequential()
cnn_lstm.add(keras.layers.TimeDistributed(vision_model,input_shape=(1,input_im_size,input_im_size, 3)))
encoded_image=cnn_lstm(image_input)
 
encoder_inputs = keras.layers.Input(shape=( 1,None  ))
encoder_embedding = keras.layers.TimeDistributed(keras.layers.Embedding( num_eng_tokens, 128 , mask_zero=True ) )(encoder_inputs)
pre_lstm=keras.layers.TimeDistributed(keras.layers.LSTM( 64  ))(encoder_embedding)
merged=keras.layers.concatenate([encoded_image,pre_lstm])
encoder_outputs , state_h , state_c = keras.layers.LSTM( 64 , return_state=True  )(merged)
encoder_states = [ state_h , state_c ]
 
decoder_inputs = keras.layers.Input(shape=( None ,  ))
decoder_embedding = keras.layers.Embedding( num_mar_tokens, 128 , mask_zero=True) (decoder_inputs)
decoder_lstm = keras.layers.LSTM( 64 ,return_state=True,return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm (decoder_embedding , initial_state=encoder_states )
decoder_dense = keras.layers.Dense( num_mar_tokens , activation=keras.activations.softmax )
output_0 = decoder_dense (decoder_outputs )
 
vqaseq_model=keras.models.Model([image_input,encoder_inputs,decoder_inputs],output_0)
 
#compile model
vqaseq_model.compile(
  optimizer=keras.optimizers.Adam(),
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)
vqaseq_model.summary()
 
vqaseq_model.fit([images,encoder_input_data , decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=num_epochs)
 
def make_inference_models():
    encoder_model = keras.models.Model([image_input,encoder_inputs], encoder_states)
 
    decoder_state_input_h = keras.layers.Input(shape=(64,))
    decoder_state_input_c = keras.layers.Input(shape=(64,))
 
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
 
    decoder_outputs_, state_h_, state_c_ = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
    decoder_states = [state_h_, state_c_]
    decoder_outputs_ = decoder_dense(decoder_outputs_)
    decoder_model = keras.models.Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs_] + decoder_states)
 
    return encoder_model, decoder_model
 
def freeze(model):
    """Freeze model weights in every layer."""
    for layer in model.layers:
        layer.trainable = False
 
        if isinstance(layer, keras.models.Model):
            freeze(layer)
    return model
 
enc_model, dec_model = make_inference_models()
 
freeze(enc_model).save('enc_model.h5')
freeze(dec_model).save('dec_model.h5')

In [ ]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import cv2
import os
import warnings
import sys
sys.stdout.write("\033[1;34m")
warnings.filterwarnings("ignore")

test_path_im='/content/drive/My Drive/test_img'
dir_path_conversation = '/content/drive/My Drive/chatterbot_dataset_ita'
test_im_len=len([name for name in os.listdir(test_path_im) if os.path.isfile(os.path.join(test_path_im, name))])
len_data_vqa=2000
input_im_size=64

#questions and answers
pre_ques = ['che cosa vedi']
pre_answers = ['vedo una camera da letto', 'vedo una città con dei palazzi', 'ci sono delle persone ',
               'una foresta con degli alberi', 'sono davanti ad una montagna', 'questo potrebbe essere un cane',
               'un gatto', 'il traffico con delle macchine', 'vedo una scrivania', 'questa dovrebbe essere una sedia']
questions = []
answers = []

for i in range(len_data_vqa):
    questions.append(pre_ques[0])

for i in range(int(len_data_vqa / 200)):
    for n in range(200):
        answers.append(pre_answers[i])

for filepath in os.listdir(dir_path_conversation):
    stream = open(dir_path_conversation + os.sep + filepath, 'r', encoding='utf-8')
    text = stream.readlines()
    for i in range(int(len(text) / 2) - 6):
        questions.append(text[i + i].rstrip('\n'))
        answers.append(text[i + i + 1].rstrip('\n'))

#questions
questions_lines = questions

tokenizer = Tokenizer(oov_token=1)
tokenizer.fit_on_texts( questions_lines )
tokenized_eng_lines = tokenizer.texts_to_sequences( questions_lines )

length_list = []
for token_seq in tokenized_eng_lines:
    length_list.append( len( token_seq ))
max_input_length = np.amax( length_list )
print( 'questions max length is {}'.format( max_input_length ))

padded_eng_lines = pad_sequences( tokenized_eng_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( np.expand_dims(padded_eng_lines, 1))
print( 'Encoder input data shape -> {}'.format( encoder_input_data.shape ))

eng_word_dict = tokenizer.word_index
num_eng_tokens = len( eng_word_dict )+1
print( 'Number of questions tokens = {}'.format( num_eng_tokens))

#answers
lines_answers=answers
answer_lines = []
for line in range( len(lines_answers)):
    answer_lines.append( '<START> ' + str(lines_answers[line]) + ' <END>' )

tokenizer_a = Tokenizer()
tokenizer_a.fit_on_texts( answer_lines )
tokenized_mar_lines = tokenizer_a.texts_to_sequences( answer_lines )

length_list = []
for token_seq in tokenized_mar_lines:
    length_list.append( len( token_seq ))
max_output_length = np.amax( length_list )
print( 'Answers max length is {}'.format( max_output_length ))

padded_mar_lines = pad_sequences( tokenized_mar_lines , maxlen=max_output_length, padding='post' )
decoder_input_data = np.array( padded_mar_lines )
print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))

mar_word_dict = tokenizer_a.word_index
num_mar_tokens = len( mar_word_dict )+1
print( 'Number of Answers tokens = {}'.format( num_mar_tokens))

#load model
enc_model=keras.models.load_model('/content/enc_model.h5',compile=False)
dec_model=keras.models.load_model('/content/dec_model.h5',compile=False)

print('Test su seq2se2_cnn avviato...')

while True:
 number_file=input('Scegliere il numero del file:')
 try:
   if int(number_file)>test_im_len or int(number_file)<1:
     print('File non trovato, riprova.')
   else:
     image=cv2.resize(cv2.imread(test_path_im+'/'+ str(number_file)+ '.jpg'),(input_im_size,input_im_size))/255.0
     image=np.expand_dims([image],1)
     question=input('Tu:')
     question=pad_sequences(tokenizer.texts_to_sequences([question]), maxlen=max_input_length , padding='post')
     question=np.expand_dims(question,1)
     states_values = enc_model.predict([image,question])
     empty_target_seq = np.zeros((1, 1))
     empty_target_seq[0, 0] = mar_word_dict['start']
     stop_condition = False
     decoded_translation = ''
     while not stop_condition:
       dec_outputs, h, c = dec_model.predict([empty_target_seq] + states_values)
       sampled_word_index = np.argmax(dec_outputs[0, -1, :])
       sampled_word = None
       for word, index in mar_word_dict.items():
         if sampled_word_index == index:
           decoded_translation += ' {}'.format(word)
           sampled_word = word

       if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
         stop_condition = True

       empty_target_seq = np.zeros((1, 1))
       empty_target_seq[0, 0] = sampled_word_index
       states_values = [h, c]

     print('AGI:' + decoded_translation.replace('end',''))
 except ValueError:
   print("Si accettano solo numeri.")